In [1]:
import numpy as np
import pandas as pd
import json
import os

In [2]:
flight_df = pd.read_csv('merged_flight_data.csv')

In [3]:
flight_df.tail()

,Year,Quarter,Month,DayofMonth,FlightDate,Origin,Dest,CRSDepTime,DepTime,DepDelayMinutes,DepDel15,CRSArrTime,ArrTime,ArrDelayMinutes,ArrDel15
11292274,2017,3,9,25,2017-09-25,ORD,BOS,1018,1332.0,194.0,1.0,1335,1647.0,192.0,1.0
11292275,2017,3,9,25,2017-09-25,JFK,LGB,1828,1822.0,0.0,0.0,2135,2135.0,0.0,0.0
11292276,2017,3,9,25,2017-09-25,JAX,FLL,630,623.0,0.0,0.0,743,732.0,0.0,0.0
11292277,2017,3,9,25,2017-09-25,FLL,JAX,700,654.0,0.0,0.0,812,802.0,0.0,0.0
11292278,2017,3,9,25,2017-09-25,PBI,BOS,1001,952.0,0.0,0.0,1301,1247.0,0.0,0.0


In [4]:
weather_dir = "weather"

columns_from_weather = ['windspeedKmph','winddirDegree','weatherCode','precipMM','visibility','pressure','cloudcover','DewPointF','WindGustKmph','tempF','WindChillF','humidity']


In [5]:
def get_json_filename(year, month):
    return f"{year}-{month}.json"

In [6]:
def format_departure_time(departure_time):
    if pd.isna(departure_time):
        return "0000"  
    else:
        return f"{int(departure_time // 100):02d}{int(departure_time % 100):02d}"


In [9]:
flight_data_with_weather = []
weather_info_lists = {col: [] for col in columns_from_weather}

In [10]:
city_list=['ATL' ,'CLT', 'DEN', 'DFW', 'EWR', 'IAH', 'JFK', 'LAS', 'LAX', 'MCO','MIA', 'ORD' ,'PHX', 'SEA', 'SFO']

In [11]:
for index, row in flight_df.iterrows():
    print(index)
    
    city_name = row['Origin']
    year = row['Year']
    month = row['Month']
    day = row['DayofMonth']
    departure_time = row['DepTime']
    dest_city = row['Dest']

    if city_name not in city_list or dest_city not in city_list:
        for col in columns_from_weather:
            weather_info_lists[col].append(np.nan)
        continue

    formatted_departure_time = format_departure_time(departure_time)

 
    json_file = os.path.join(weather_dir, city_name, get_json_filename(year, month))

    if os.path.exists(json_file):
        with open(json_file, 'r') as f:
            data = json.load(f)
            hourly_data = data['data']['weather'][day-1]['hourly']
            closest_hour = min(hourly_data, key=lambda x: abs(int(x['time'][0:2]) - int(formatted_departure_time[:2])))
            weather_info = {key: closest_hour[key] for key in columns_from_weather}
            for col, value in weather_info.items():
                weather_info_lists[col].append(value)



KeyboardInterrupt: 

In [ ]:
for col in weather_info_lists:
    flight_df[col] = weather_info_lists[col]